In [2]:
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import networkx as nx
import ndlib.models.ModelConfig as mc
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import os
import pickle
import sys
import future.utils
import matplotlib as mpl
import matplotlib.pyplot as plt 
from matplotlib import rc
mpl.rcParams["text.usetex"] = True
plt.rc('font',**{'family':'serif', 'size' : 22, 'serif':['Computer Modern Roman']})
import os
import sys
import future.utils
import warnings
# warnings.filterwarnings("ignore")
from datetime import datetime
from pathlib import Path
import pickle
import abc
import numpy as np
import past.builtins
import six
import tqdm
import networkx as nx
import csv
from sklearn.cluster import MeanShift
import ndlib.models.ModelConfig as mc
from ndlib.viz.mpl.OpinionEvolution import OpinionEvolution
import ndlib.models.opinions as op
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
%matplotlib inline

no display found. Using non-interactive Agg backend


In [3]:
sns.set_style("whitegrid")

# Spaghetti plot

In [17]:
class OpinionEvolution(object):
    
    def __init__(self, model, trends):
        """
        :param model: The model object
        :param trends: The computed simulation trends
        """
        self.model = model
        self.srev = trends
        self.ylabel = "Opinion"
    
    def plot(self, fig, ax, filename=None):
        nodes2opinions = {}
        node2col = {}

        last_it = self.srev[-1]['iteration'] + 1
        last_seen = {}

        for it in self.srev:
            sts = it['status']
            its = it['iteration']
            for n, v in sts.items():
                if n in nodes2opinions:
                    last_id = last_seen[n]
                    last_value = nodes2opinions[n][last_id]

                    for i in range(last_id, its):
                        nodes2opinions[n][i] = last_value

                    nodes2opinions[n][its] = v
                    last_seen[n] = its
                else:
                    nodes2opinions[n] = [0]*last_it
                    nodes2opinions[n][its] = v
                    last_seen[n] = 0
                    if v < 0.4:
                        node2col[n] = '#3776ab'
                    elif 0.4 <= v <= 0.6:
                        node2col[n] = '#FFA500'
                    else:
                        node2col[n] = '#FF0000'       
        mx = 0
        for k, l in future.utils.iteritems(nodes2opinions):
            if mx < last_seen[k]:
                mx = last_seen[k]
            x = list(range(0, last_seen[k]))
            y = l[0:last_seen[k]]
            ax.plot(x, y, lw=1, alpha=0.5, color=node2col[k])

        ax.set_ylim(-0.1, 1.1)
        ax.yaxis.set_minor_locator(MultipleLocator(0.1))
        ax.tick_params(axis='both', which='major', labelsize=3, pad=2)                
        plt.grid(axis = 'both', which='both')
        plt.tight_layout()


In [18]:
def restoiterations(filename):   
    iterations = list()
    try:
        infile = open(filename, "r")
        lines = infile.readlines()
        for i,line in enumerate(lines):
            opinions = list(line.split(','))
            itdict = {"iteration":i, "status":{k: float(v) for k, v in enumerate(opinions)},"node_count":{0:0},"status_delta":{0:0}}
            iterations.append(itdict)
    except FileNotFoundError:
        pass
    return iterations

In [30]:
def spaghetti(e,p,g,fig,ax,quale=None):
    try:
        graph = nx.complete_graph(100)
        model = op.AlgorithmicBiasMediaModel(graph) 
        config = mc.Configuration()
        config.add_model_parameter("p", p)
        config.add_model_parameter("k", 3)
        config.add_model_parameter("epsilon", e)
        config.add_model_parameter("gamma", g)
        config.add_model_parameter("gamma_media",g)
        model.set_initial_status(config)
        model.set_media_opinions([0.05, 0.5, 0.95])
    except Exception as err:
        pass
    nr=1
    while True:
        iterations = restoiterations("res/onemedia/media mo[0.0] p{} e{} g{} gm{} mi1000000 nr{}".format(p,e,g,g,nr))
        if len(iterations)>0:
            viz = OpinionEvolution(model, iterations)
            viz.plot(fig, ax)
            break
        else:
            nr+=1


In [31]:
def spaghettigrid(p):
    fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(6, 3), dpi=600, sharey=True)
    row=0
    for e in [0.2, 0.3, 0.5]:
        col=0
        for g in [0.0, 0.5, 0.75, 1.0, 1.5]: 
            spaghetti(e,p,g,fig,ax=axes[row,col],quale='onemedia')
            col+=1
            pbar.update(1)
        row+=1

In [32]:
mpl.rcParams["text.usetex"] = True
plt.rc('font',weight='bold',**{'family':'serif', 'size':6, 'serif':['Computer Modern Roman']})
for p in [0.1, 0.3, 0.5]:
    spaghettigrid(p=p)